In [0]:
!pip install kaggle


### Head to kaggle and download your dataset
refer to this [Youtube video](https://www.youtube.com/watch?v=eEgZtNOCJhk) on downloading kaggle dataset.
After downloading upload your kaggle.json to jupyter workspace 

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#change permission
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [0]:
!unzip chest-xray-pneumonia.zip

In [0]:
!unzip chest_xray.zip

In [0]:
PATH = "../pnemonia-detection/chest_xray/train"

In [0]:
!cd ../content/chest_xray/train/

In [0]:
!ls

In [0]:
from os import path
path.exists(PATH)

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.vision import *
from fastai.metrics import accuracy

In [0]:
data = ImageDataBunch.from_folder(PATH, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, bs = 64).normalize(imagenet_stats)

In [0]:
data.show_batch(rows = 2, fig = (5,5))

In [0]:
class NotificationCallback(Callback):
    """
        PyTorch callback for model training
    """
    def on_train_begin(self, metrics_names: StrList, **kwargs: Any) -> None:
        send_notification("Training has Begun ")

        
    def on_epoch_end(self,  epoch: int, smooth_loss: Tensor, last_metrics: MetricsList, **kwargs: Any) -> None:
        super().on_epoch_end(**kwargs)
        val_loss, accuracy = last_metrics[0], last_metrics[1]
        message = "Epoch: " +str(epoch+1) + "-> " +"Train loss: " +f"{smooth_loss.item():.4f}" + " , " +" Val loss: "+ f"{val_loss:.4f}" + " , " + " Accuracy: " +f"{accuracy:.4f}"
        send_notification(message)
        
    def on_train_end(self, metrics, **kwargs: Any) -> None:
        super().on_train_end(**kwargs)
        send_notification("Training finished ")

####Head over to https://pushover.net and get your api key and download mobile app too

In [0]:
def send_notification(msg):
    """
        Send message to mobile using Pushover notifications.
        Calls Pushover API to do that.
        Pushover API docs: https://pushover.net/api
    """
    import requests
    from datetime import datetime

    url = "https://api.pushover.net/1/messages.json"
    data = {
        "user"  : "SECRET",
        "token" : "SECRET",
        "sound" : "cashregister"
    }
    data["message"] = msg
    data['message'] = data['message'] + "\n" + str(datetime.now())

    r =  requests.post(url = url, data = data)

In [0]:
notif_cb = NotificationCallback()

### Create Learner


In [0]:
learn = cnn_learner(data, models.resnet50, metrics=[accuracy,error_rate] , model_dir="/tmp/model/",callbacks = [notif_cb])

### If everything works well you should be getting notifs on your mobile phone on running below cell

In [0]:
learn.fit_one_cycle(4)

In [0]:
learn.recorder.plot_metrics()

### Save the plot to be sent as mobile notifications

In [0]:
import matplotlib
import matplotlib.pyplot as plt
learn.recorder.plot_losses()

plt.savefig('lossplot.png')

In [0]:
PATH_TO_IMAGE = "../pnemonia-detection/lossplot.png"

## Running below cell send the loss plot to your mobile phone as notification
Insert your token and user key generated from pushover below

In [0]:
import requests
r = requests.post("https://api.pushover.net/1/messages.json", data={"token":"SECRET KEY","user":"SECRET KEY","message":"Loss Plots"}, files={"attachment":open(PATH_TO_IMAGE,"rb")})

In [0]:
learn.save('stage-1')

### Confusion matrix and top losses

In [0]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

In [0]:
interp.plot_top_losses(9, figsize=(15,11))

In [0]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)